In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import hvplot.pandas  # noqa
from functools import cache
from constants import *
from bokeh.plotting import figure, show, output_notebook
output_notebook()


test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE

# Get nifty candles for minute
# For first minute, get premium strike price for call and puts which is price at ~30 
# Draw them here 
# Try to match the pattern 

/home/nitin/nse-reports-trade/utils.py:39: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  scrip_df = pd.read_csv(file_path)


Loading BokehJS ...

In [2]:
nifty_candles = ut.get_data(symbol=SYMBOL, date=test_date, interval=INTERVAL, exchange=EXCHANGE)
atm_strike = 21700
otm_call_strike = 22100
otm_put_strike = 21300
expiry = ut.find_nclosest_expiry(SYMBOL, test_date, 1)

call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=expiry, cur_dt=test_date.date(), strike_price=otm_call_strike, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=expiry, cur_dt=test_date.date(), strike_price=otm_put_strike, option_type=OPTION_TYPE_PUT)

In [7]:
# Create a vbar chart
# Create a box plot 
# Merge them together and come up with a good candlestick chart
from bokeh.models import ColumnDataSource, BooleanFilter, CDSView, IndexFilter, CrosshairTool, HoverTool, WheelZoomTool, DatetimeTickFormatter, DatetimeTicker, TickFormatter, CustomJSTickFormatter, Label, CustomJS, NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.io import curdoc
from bokeh.themes import Theme


def create_candlestick_plot(df):
    cds = ColumnDataSource(df)
    green = CDSView(filter=BooleanFilter(df['close'] >= df['open']))
    red = CDSView(filter=BooleanFilter(df['close'] < df['open']))
    w = 60*1000
    p = figure(
        x_axis_type="datetime",
        title=f"Minute Candles",
        min_width=2000,
        min_height=900,
        background_fill_color="#1e1e1e"
    )
    
    # Segments for high-low
    p.segment(x0='date_time', y0='high', x1='date_time', y1='low', color="#26a69a", source=cds, view=green)
    p.segment(x0='date_time', y0='high', x1='date_time', y1='low', color="#ef5350", source=cds, view=red)
    
    # Bars for open-close
    p.vbar(x='date_time', width=w, top='open', bottom='close', fill_color="#26a69a", line_color="black", source=cds, view=green)
    p.vbar(x='date_time', width=w, top='close', bottom='open', fill_color="#ef5350", line_color="black", source=cds, view=red)
    
    # p.xaxis.formatter = DatetimeTickFormatter(
    #     minutes="%H:%M",
    #     hours="%H:%M",
    #     days="%H:%M",
    #     months="%H:%M",
    #     years="%H:%M"
    # )
    
    p.xaxis.formatter = CustomJSTickFormatter(code="""
        var date = new Date(tick);
        var hours = date.getUTCHours();
        var minutes = date.getUTCMinutes();
        var suffix = (hours >= 12) ? 'PM' : 'AM';
        hours = (hours % 12) || 12;
        minutes = minutes < 10 ? '0' + minutes : minutes;
        return hours + ':' + minutes;
    """)
    
    p.xaxis.ticker = DatetimeTicker(
        desired_num_ticks=30,
        num_minor_ticks=5
    )
    
    p.xaxis.major_label_orientation = 0.5
    p.grid.grid_line_alpha = 0.3
    
    crosshair_tool = CrosshairTool(
        dimensions="both",
        line_color="red",
        line_alpha=0.8,
    )
    p.add_tools(crosshair_tool)
    hover = HoverTool(tooltips=[("Date", "@date_time{%H:%M}"), ("Open", "@open"), ("High", "@high"), ("Low", "@low"), ("Close", "@close"), ("Volume", "@volume{0.0a}"), ], formatters={'@date_time': 'datetime'})
    p.add_tools(hover)
    wheel_zoom = WheelZoomTool()
    p.add_tools(wheel_zoom)
    p.toolbar.active_scroll = wheel_zoom
    
    
    tradingview_theme = Theme(json={
        'attrs': {
            'figure': {
                'background_fill_color': "#1e1e1e",
                'border_fill_color': "#1e1e1e",
                'outline_line_color': "#393939"
            },
            'Axis': {
                'major_label_text_color': "#e0e0e0",
                'axis_label_text_color': "#e0e0e0",
                'major_tick_line_color': "#393939",
                'minor_tick_line_color': "#393939",
                'axis_line_color': "#393939"
            },
            'Grid': {
                'grid_line_color': "#393939"
            }
        }
    })
    
    
    # Create labels for crosshair values on the axes
    x_label = Label(x=0, y=0, x_units='data', y_units='screen', text='', text_color='white',
                    text_font_size='10pt', background_fill_color='#1e1e1e',
                    background_fill_alpha=0.8, text_align='left', text_baseline='bottom')
    
    y_label = Label(x=0, y=0, x_units='screen', y_units='data', text='', text_color='white',
                    text_font_size='10pt', background_fill_color='#1e1e1e',
                    background_fill_alpha=0.8, text_align='left', text_baseline='bottom')
    
    p.add_layout(x_label, 'below')
    p.add_layout(y_label, 'below')
    
    # CustomJS callback to update labels
    callback = CustomJS(args={'x_label': x_label, 'y_label': y_label, 'plot': p}, code="""
        const { x, y } = cb_data['geometry'];
        const { sx, sy } = cb_data['geometry'];
        const plotHeight = plot.height;
        if (sx !== undefined || sy !== undefined) {
            const date = new Date(x);
            var hours = date.getUTCHours();
            var minutes = date.getUTCMinutes();
            hours = (hours % 12) || 12;
            minutes = minutes < 10 ? '0' + minutes : minutes;
            const xval = hours + ':' + minutes;
    
            const yValue = y.toFixed(2);
    
            x_label.x =  x;
            x_label.y = 0;  // Slightly offset from the bottom
            x_label.text = xval;
    
            y_label.x = 0;  // Slightly offset from the left
            y_label.y = y;
            y_label.text = yValue;
    
            x_label.visible = true;
            y_label.visible = true;
        }
    """)
    
    # Add hover tool to update labels
    nhover = HoverTool(tooltips=None)
    
    nhover.callback = callback
    
    p.add_tools(nhover)
    
    curdoc().theme = tradingview_theme
    
    volume_fig = figure(
        x_axis_type="datetime",
        title="Volume",
        min_width=2000,
        height=250,  # Adjust height as needed
        background_fill_color="#1e1e1e",
        x_range=p.x_range,
    )
    
    # Bars for volume
    volume_fig.vbar(x='date_time', width=w, top='volume', fill_color="#26a69a", line_color="black", source=cds, view=green)
    volume_fig.vbar(x='date_time', width=w, top='volume', fill_color="#ef5350", line_color="black", source=cds, view=red)
    
    volume_fig.xaxis.formatter = CustomJSTickFormatter(code="""
        var date = new Date(tick);
        var hours = date.getUTCHours();
        var minutes = date.getUTCMinutes();
        var suffix = (hours >= 12) ? 'PM' : 'AM';
        hours = (hours % 12) || 12;
        minutes = minutes < 10 ? '0' + minutes : minutes;
        return hours + ':' + minutes;
    """)
    
    volume_fig.xaxis.ticker = DatetimeTicker(
        desired_num_ticks=30,
        num_minor_ticks=5
    )
    
    volume_fig.yaxis.formatter = NumeralTickFormatter(format='0.0a')
    
    volume_fig.xaxis.major_label_orientation = 0.5
    volume_fig.grid.grid_line_alpha = 0.3
    
    volume_fig.add_tools(crosshair_tool)
    volume_fig.add_tools(hover)
    volume_fig.add_tools(wheel_zoom)
    volume_fig.toolbar.active_scroll = wheel_zoom
    
    
    layout = gridplot([[p], [volume_fig]])
    show(layout)
    return layout

create_candlestick_plot(call_df)

GridPlot(id='p1630', ...)

In [3]:
ut.create_candlestick_plot(put_df)

GridPlot(id='p1202', ...)

In [5]:
call_df

,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4
date_time,,,,,,,,,
2024-01-01 09:15:00,22.00,24.40,16.95,17.20,543200,2874400,18.50,17.20,17.30
2024-01-01 09:16:00,17.05,17.65,16.50,17.00,390900,2874400,17.91,16.95,17.00
2024-01-01 09:17:00,17.10,18.20,17.00,17.20,249000,2925900,17.83,17.15,17.25
2024-01-01 09:18:00,17.35,17.55,16.55,16.75,336450,2925900,17.64,16.75,16.80
2024-01-01 09:19:00,16.80,16.80,15.85,16.05,268450,2925900,17.45,16.00,16.05
...,...,...,...,...,...,...,...,...,...
2024-01-01 15:25:00,10.90,11.05,9.95,10.40,731700,4262250,20.87,10.40,10.45
2024-01-01 15:26:00,10.40,10.85,10.35,10.50,297100,4262250,20.81,10.50,10.55
2024-01-01 15:27:00,10.50,10.85,10.50,10.75,449150,4274850,20.74,10.75,10.80


In [6]:
put_df

,open,high,low,close,volume,unknown1,unknown2,unknown3,unknown4
date_time,,,,,,,,,
2024-01-01 09:15:00,17.05,19.35,7.35,17.80,235500,2881650,17.78,17.65,17.75
2024-01-01 09:16:00,17.60,17.95,16.55,17.75,192000,2881650,17.56,17.70,17.75
2024-01-01 09:17:00,17.50,17.50,16.35,16.75,274150,2930050,17.27,16.75,16.80
2024-01-01 09:18:00,16.40,16.70,15.95,16.15,280700,2930050,17.01,16.10,16.20
2024-01-01 09:19:00,16.00,16.55,15.60,16.45,123000,2930050,16.89,16.40,16.45
...,...,...,...,...,...,...,...,...,...
2024-01-01 15:25:00,11.15,11.20,10.25,10.85,364550,3019950,9.99,10.85,10.90
2024-01-01 15:26:00,10.90,11.30,10.80,10.90,190650,3019950,9.99,10.85,10.90
2024-01-01 15:27:00,10.80,11.00,10.40,10.40,435600,3015000,10.01,10.35,10.45
